In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ip3fg59e/unsloth_9912315a9cc349e6b24d50b0c6601a47
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ip3fg59e/unsloth_9912315a9cc349e6b24d50b0c6601a47
  Resolved https://github.com/unslothai/unsloth.git to commit 2f2b478868f63b66aaaa93db66ab3d811cddc95e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

We are going to use unsloth which is is a lightweight library for faster LLM fine-tuning which is fully compatible with the Hugging Face ecosystem.

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


We will load our phi-3 model in 4bit precision since 8 bit is creating problem.

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. 8bit wasnt'available


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: You passed in `unsloth/Phi-3-mini-4k-instruct` and `load_in_4bit = True`.
We shall load `unsloth/Phi-3-mini-4k-instruct-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We are going to give prompt to the model which is not finetuned and want to see what result it is going to generate.
As we can see it is not generating script in a way we want. Thats why we are going to finetune this model on a synthetic dataset.

In [ ]:
alpaca_prompt = """You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line. But don't generate the hook ,generated text must only be script part.

### Hook:
{}

### Script:
{}"""
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "If you are the one who dreams to get into top tech companies but do not have any technical background.I got you....", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

<s> You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
If you are the one who dreams to get into top tech companies but do not have any technical background.I got you....

### Script:

[Opening shot of a bustling tech company's office]

Narrator (Voiceover): "In the heart of Silicon Valley, where innovation thrives, lies the dream of every aspiring tech professional - to land a coveted position at a top tech company. But what if you don't have a technical background?"

[Cut to a montage of people from various backgrounds entering the tech company]

Narrator (Voiceover): "Imagine a world where your passion for technology, combined with your unique skills and experiences, can open doors to the most prestigious tech companies."

[Trans

## Fine-Tune the Model with LoRA (LoRa Optimization)**

In this step, we fine-tune the loaded model using LoRA (Low-Rank Attention) optimization techniques. This optimization method improves the efficiency and memory usage of the model, making it suitable for large-scale language modeling tasks.

### **Key Parameters for Fine-Tuning**

- `r`: The low-rank factorization parameter. Choose any number greater than 0. Recommended values are 8, 16, 32, 64, or 128. This parameter controls the degree of factorization in the attention layers.
- `target_modules`: Specifies the attention modules to apply LoRA to. These include "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", and "down_proj". You can customize this list based on your requirements.
- `lora_alpha`: The alpha parameter for LoRA. This parameter controls the degree of regularization in the low-rank approximation. A higher value results in stronger regularization.
- `lora_dropout`: Dropout probability for LoRA. While any dropout value is supported, setting it to 0 is optimized for performance.
- `bias`: Specifies the type of bias to use in the attention layers. Setting it to "none" optimizes memory usage.
- `use_gradient_checkpointing`: Controls whether to use gradient checkpointing to reduce memory usage. Setting it to "unsloth" enables an optimized mode that reduces VRAM usage by 30% and supports larger batch sizes.
- `random_state`: Seed for random number generation to ensure reproducibility.

### **Fine-Tuning Process**

We utilize the `get_peft_model` method from the `FastLanguageModel` API to apply LoRA optimization to the model. This process enhances the model's performance and efficiency, making it well-suited for various NLP tasks.



## **Step 3: Apply LoRA Optimization to the Model**

In this step, we apply LoRA (Low-Rank Attention) optimization to the fine-tuned model. LoRA improves the efficiency and memory usage of the model's attention mechanisms, enhancing its performance for large-scale language modeling tasks.

### **Key Parameters for LoRA Optimization**

- `r`: The low-rank factorization parameter. Choose any number greater than 0. Recommended values are 8, 16, 32, 64, or 128. This parameter controls the degree of factorization in the attention layers.
- `target_modules`: Specifies the attention modules to apply LoRA to. These include "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", and "down_proj". You can customize this list based on your requirements.
- `lora_alpha`: The alpha parameter for LoRA. This parameter controls the degree of regularization in the low-rank approximation. A higher value results in stronger regularization.
- `lora_dropout`: Dropout probability for LoRA. While any dropout value is supported, setting it to 0 is optimized for performance.
- `bias`: Specifies the type of bias to use in the attention layers. Setting it to "none" optimizes memory usage.
- `use_gradient_checkpointing`: Controls whether to use gradient checkpointing to reduce memory usage. Setting it to "unsloth" enables an optimized mode that reduces VRAM usage by 30% and supports larger batch sizes.
- `random_state`: Seed for random number generation to ensure reproducibility.
- `use_rslora`: Option to enable rank-stabilized LoRA. Not enabled in this example.
- `loftq_config`: LoftQ configuration. Not enabled in this example.

### **LoRA Optimization Process**

We utilize the `get_peft_model` method from the `FastLanguageModel` API to apply LoRA optimization to the fine-tuned model. This process enhances the model's performance and efficiency, making it well-suited for various NLP tasks.




In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 6, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 12,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
model.print_trainable_parameters()

trainable params: 11,206,656 || all params: 3,832,286,208 || trainable%: 0.2924


We can see that we only have to train nearly 11M parameters and not the whole 3.8 billion patrams using above lora configuration

In [ ]:
import pandas as pd
URL  = "https://docs.google.com/spreadsheets/d/1D7pW-i964C8zYVTQdH7m0hsNDu7espezgp47qYoGcNU/edit#gid=0"

sheet_url = URL.replace("/edit#gid=", "/export?format=csv&gid=")
df =pd.read_csv(sheet_url)
df.columns

Index(['Date ', 'Reference URL', 'Hook', 'Build Up', 'Body ', 'CTA'], dtype='object')

In [ ]:
print(tokenizer.eos_token)

<|endoftext|>


## **Preparing Alpaca-Prompt Formatted Dataset**

Here I created the format for the datset to feed into phi-3 model for finetuning in which i specifically instructed the model to be acting as scriptwrite and match the style of generating script same ass the dataset


In [ ]:
import pandas as pd
from datasets import Dataset

# dataset_url = "https://docs.google.com/spreadsheets/d/1D7pW-i964C8zYVTQdH7m0hsNDu7espezgp47qYoGcNU/edit#gid=0"

# Define the Alpaca-Prompt template
alpaca_prompt = """You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
{}

### Script:
{}"""

EOS_TOKEN = tokenizer.eos_token
# Function to format prompts
def formatting_prompts_func(row):
    hook = row["Hook"]
    build_up = row["Build Up"]
    body = row["Body "]
    cta = row["CTA"]
    texts = []
    for hook, build_up,body,cta  in zip(hook, build_up,body,cta ):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        prompt = alpaca_prompt.format(hook, build_up+" "+body +" "+ cta) +EOS_TOKEN
        texts.append(prompt)
    return { "text" : texts, }

    # return prompt

hf_dataset = Dataset.from_pandas(df)

# Apply the formatting function to the dataset
hf_dataset = hf_dataset.map(formatting_prompts_func, batched=True, batch_size=len(df))

# Remove unnecessary columns (if any)
hf_dataset = hf_dataset.remove_columns(['Date ', 'Reference URL', 'Hook', 'Build Up', 'Body ', 'CTA'])

# Check the first few examples
print(hf_dataset['text'][:3])

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

["You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.\n\n### Hook:\n:Multiple big creators stole my content and got millions of views, more views than even me, the original creator.\n\n### Script:\nMultiple big creators stole my content and got millions of views, more views than even me, who is the original. Now, I could be mad at them, but I'm a giver and I will give you two of the best ways to find wild content they shouldn't steal, but use as inspiration. Multiple big creators stole my content and got millions of views, more views than even me, who is the original. Now, I could be mad at them, but I'm a giver and I will give you two of the best ways to find wild content they shouldn't steal, but use as inspiration. First is Tweet Hunter. Fin

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 75,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/43 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.688 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 43 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 75
 "-____-"     Number of trainable parameters = 11,206,656


Step,Training Loss
1,2.506300
2,2.688300
3,2.722600
4,2.419900
5,2.490900
6,2.531200
7,2.227300
8,2.207500
9,2.295300
10,2.201600


In [ ]:
trainer_stats.metrics


{'train_runtime': 372.5869,
 'train_samples_per_second': 1.61,
 'train_steps_per_second': 0.201,
 'total_flos': 4837753988352000.0,
 'train_loss': 1.5819990984598795,
 'epoch': 13.636363636363637}

### Now its time to generate some cool scripts

As you can observe the finetuned model is generating output as expected and is capable of generating script for video content.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "So you want to learn how to to capture these type of stunning photographs. I have a good news for you ", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

<s> You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
So you want to learn how to to capture these type of stunning photographs. I have a good news for you 

### Script:
This video is for all the photographers out there who want to learn how to take stunning photos like this. And the good news is that there's a simple formula that you can use to take stunning photos. And the bad news is that you're not using it right now. The formula is this: Subject + Light + Color. And I'm going to break down each of these elements for you right now. So the first element is the subject. And the subject is the focal point of your photo. It's what you want people to look at. And in this photo, the subject is the girl. The second element is light. A

### It is pretty much obvious that now the model learnt from the dataset the style in which we want our script :)

### Prompt:
"So you want to learn how to to capture these type of stunning photographs. I have a good news for you "
### Generated script:
This video is for all the photographers out there who want to learn how to take stunning photos like this. And the good news is that there's a simple formula that you can use to take stunning photos. And the bad news is that you're not using it right now. The formula is this: Subject + Light + Color. And I'm going to break down each of these elements for you right now. So the first element is the subject. And the subject is the focal point of your photo. It's what you want people to look at. And in this photo, the subject is the girl. The second element is light. And light is everything in photography. It's what separates a good photo from a great photo. And in this photo, the light is coming from the left. And the third element is color. And color is everything in photography. It's what separates a good photo from a great photo. And in this photo, the color is warm. And if you want to learn how to take stunning photos like this, just follow the formula. Subject + Light + Color.<|endoftext|>

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "If you are the one who dreams to get into top tech companies but do not have any technical background.I got you....", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)


<s> You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
If you are the one who dreams to get into top tech companies but do not have any technical background.I got you....

### Script:
This is how you can get into top tech companies like Google, Facebook, Apple, Microsoft, etc. without having any technical background. The secret is to learn to code and build something awesome. The first step is to learn to code. You can learn to code in 2021 by following the path that I am going to share with you. First, you need to learn the fundamentals of programming by taking a course on freeCodeCamp. Once you have learned the fundamentals, you need to build a personal website using HTML, CSS, and JavaScript. This will help you learn how to creat

### Prompt:
"If you are the one who dreams to get into top tech companies but do not have any technical background.I got you.... "
### Generated script:
This is how you can get into top tech companies like Google, Facebook, Apple, Microsoft, etc. without having any technical background. The secret is to learn to code and build something awesome. The first step is to learn to code. You can learn to code in 2021 by following the path that I am going to share with you. First, you need to learn the fundamentals of programming by taking a course on freeCodeCamp. Once you have learned the fundamentals, you need to build a personal website using HTML, CSS, and JavaScript. This will help you learn how to create a user interface and user experience. Next, you need to learn how to use a static site generator called JAMstack to build a professional-looking website. This will help you learn how to create a scalable and secure website. Finally, you need to learn how to use a content management system (CMS) like Gatsby to build a dynamic and engaging website. This will help you learn how to create a website that is easy to maintain and update. By following this path, you will learn all the skills that top tech companies are looking for in their candidates. You will also have a portfolio of work that you can showcase to these companies. So, if you want to get into top tech companies without having any technical background, follow this path and start building your future today.<|endoftext|>

In [ ]:

def generate_script(hook):
    FastLanguageModel.for_inference(model)
    prompt = alpaca_prompt.format(hook, "")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # text_streamer = TextStreamer(tokenizer)
    outputs = model.generate(**inputs, max_new_tokens=1000)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text



# Example usage
hook = "Ready to get fit and strong? In today’s video, we’ll take you through a full-body workout that you can do at home with no equipment. Let’s get started on your fitness journey!"
print(generate_script(hook))

You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
Ready to get fit and strong? In today’s video, we’ll take you through a full-body workout that you can do at home with no equipment. Let’s get started on your fitness journey!

### Script:
Building muscle and getting fit is all about consistency and following a proven workout routine. In this video, we’ll go through a full-body workout that you can do at home with no equipment. This workout will help you build muscle, increase strength, and improve your overall fitness.

First, let’s warm up with some dynamic strets to get our blood flowing and our muscles ready for action.

Next, we’ll move on to the main workout, which consists of three circuits. Each circuit will target different

for video :
Lets try these prompts:
1)Looking for a new smartphone? Today, we’re reviewing the latest model from TechCorp, the UltraPhone X. With its cutting-edge features and sleek design, is this the phone you’ve been waiting for? Let’s find out.


2)Welcome to our latest travel adventure! This time, we are exploring the breathtaking landscapes and rich culture of Iceland. From majestic waterfalls to the vibrant city of Reykjavik, join us as we embark on an unforgettable journey.

In [ ]:
hook = "Looking for a new smartphone? Today, we’re reviewing the latest model from TechCorp, the UltraPhone X. With its cutting-edge features and sleek design, is this the phone you’ve been waiting for? Let’s find out."
print(generate_script(hook))

You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
Looking for a new smartphone? Today, we’re reviewing the latest model from TechCorp, the UltraPhone X. With its cutting-edge features and sleek design, is this the phone you’ve been waiting for? Let’s find out.

### Script:
The script for the content provided in the hook.

### Script:
The UltraPhone X is the latest and greatest from TechCorp. It boasts a stunning 6.8-inch AMOLED display with a 120Hz refresh rate, providing smooth and vibrant visuals. The phone is powered by the new TechCorp Titan processor, which delivers lightning-fast performance and handles multitasking like a dream.

The UltraPhone X also features a state-of-the-art 50MP camera system with advanced AI capabiliti

In [ ]:
hook = "Welcome to our latest travel adventure! This time, we are exploring the breathtaking landscapes and rich culture of Iceland. From majestic waterfalls to the vibrant city of Reykjavik, join us as we embark on an unforgettable journey."
print(generate_script(hook))

You are an expert scriptwriter for creating engaging and informative video content. Below is the context (hook) about the beginning of the script for a video. Your task is to generate a detailed and descriptive script for the content. Also provide a strong Call to Action (CTA) in a separate line.

### Hook:
Welcome to our latest travel adventure! This time, we are exploring the breathtaking landscapes and rich culture of Iceland. From majestic waterfalls to the vibrant city of Reykjavik, join us as we embark on an unforgettable journey.

### Script:
In this video, we'll be diving into the mesmerizing beauty of Iceland. We'll be visiting the iconic Gullfoss waterfall, capturing the stunning Northern Lights, and immersing ourselves in the unique Icelandic culture. We'll also be exploring the vibrant city of Reykjavik, where we'll indulge in delicious local cuisine and experience the lively nightlife. So, if you're looking for your next travel destination, Iceland should definitely be on 

In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')